<a href="https://colab.research.google.com/github/GabrielGaete/CodeChallengeGlobant/blob/main/BDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from io import StringIO
from google.cloud import bigquery
from google.oauth2 import service_account

csv_data = """transaction_id,product_name,quantity_sold,date_sold,price,store_location,user_agent
1001,Smartphone,2,2023-10-01,300,New York,"Mozilla/5.0 (Android)"
1002,Laptop,1,2023-10-02,800,Los Angeles,"Mozilla/5.0 (Windows)"
1002,Laptop,,2023-10-02,,"Mozilla/5.0 (Windows)" # Registro duplicado con valores faltantes
1003,Headphones,5,2023-10-03,50,Chicago,"Mozilla/5.0 (iOS)"
1004,Smartwatch,3,2023-10-04,150,San Francisco,"Mozilla/5.0 (Android)"
1005,Tablet,2,2023-10-05,400,Boston,"Mozilla/5.0 (macOS)"
1006,Camera,1,2023-10-06,500,Miami,"Mozilla/5.0 (Windows)"
1007,Charger,10,2023-10-07,15,Seattle,"Mozilla/5.0 (Linux)"
1008,Mouse,4,2023-10-08,25,Dallas,"Mozilla/5.0 (Android)"
1009,Keyboard,3,2023-10-09,45,Houston,""
1010,Monitor,2,2023-10-10,200,Philadelphia,"Mozilla/5.0 (macOS)"
"""

df = pd.read_csv(StringIO(csv_data))


df = df.drop_duplicates() #limpieza y transformación de datos
df = df.fillna('ValorFaltante')

resultado_sql = df.groupby('store_location')['quantity_sold'].sum()    # Supongamos que queremos segmentar por la ubicación de la tienda y calcular la suma de la cantidad vendida.

print("Resultado de la consulta SQL:")   # resultado de la consulta SQL
print(resultado_sql)

credentials = service_account.Credentials.from_service_account_file('C:\\Users\\Gabriel\\Descargas\\codechallenge-402300-20745a6f28bf.json',
                                                                    scopes=["https://www.googleapis.com/auth/cloud-platform"]) #autenticacion de Bigquery

client = bigquery.Client(credentials=credentials, project='GGaeteCodeChallenge') # cliente de BigQuery

dataset_id = 'transacciones'   #doy nombre a los conjuntos de datos y tabla de Bigquery
table_id = 'registros_ventas'

# Define el esquema de la tabla en BigQuery
schema = [
    bigquery.SchemaField("transaction_id", "INTEGER"),
    bigquery.SchemaField("product_name", "STRING"),
    bigquery.SchemaField("quantity_sold", "INTEGER"),
    bigquery.SchemaField("date_sold", "DATE"),
    bigquery.SchemaField("price", "FLOAT"),
    bigquery.SchemaField("store_location", "STRING"),
    bigquery.SchemaField("user_agent", "STRING"),
]

# Carga de datos desde el DataFrame
job_config = bigquery.LoadJobConfig(schema=schema, write_disposition="WRITE_TRUNCATE")

table_ref = client.dataset(dataset_id).table(table_id)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)

job.result()

print("Datos cargados en BigQuery.")


Resultado de la consulta SQL:
store_location
Boston                                                                        2.0
Chicago                                                                       5.0
Dallas                                                                        4.0
Houston                                                                       3.0
Los Angeles                                                                   1.0
Miami                                                                         1.0
Mozilla/5.0 (Windows) # Registro duplicado con valores faltantes    ValorFaltante
New York                                                                      2.0
Philadelphia                                                                  2.0
San Francisco                                                                 3.0
Seattle                                                                      10.0
Name: quantity_sold, dtype: object


FileNotFoundError: ignored